In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
import muon as mu
import anndata as ad
from muon import prot as pt
import squidpy as sq
import scanpy as sc
import plotnine as p9

/Users/pschafer/miniconda3/envs/misty_liana/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import liana as li
from liana.method import MistyData, genericMistyData, lrMistyData
li.__version__

'1.0.0a0'

In [3]:
# from https://www.hcdm.org/index.php/molecule-information
cd_to_ncbi = pd.read_csv("CD_to_ncbi.txt", sep="\t", header=None, names=["CD_NAME", "NCBI_NAME", "GENE_NAME", "NCBI_OTHER_NAME"])
cd_to_ncbi.NCBI_OTHER_NAME = [str(s).split(";") for s in cd_to_ncbi.NCBI_OTHER_NAME]
cd_to_ncbi

,CD_NAME,NCBI_NAME,GENE_NAME,NCBI_OTHER_NAME
0,CD1a,CD1A,CD1A,[CD1]
1,CD1b,CD1B,CD1B,[CD1]
2,CD1c,CD1C,CD1C,[CD1]
3,CD1d,CD1D,CD1D,[none]
4,CD1e,CD1E,CD1E,[HSCDIEL]
...,...,...,...,...
412,CD367,CLEC4A,CD367,"[DCIR, DDB27, CLECSF6]"
413,CD368,CLEC4D,CD368,"[MCL, CLECSF8, CLEC-6, MPCL]"
414,CD369,CLEC7A,CD369,"[DECTIN-1, CLECSF12,]"
415,CD370,CLEC9A,CD370,"[HEEE9341, UNQ9341, DNGR1]"


In [4]:
#from pybiomart import Dataset
#dataset = Dataset(name='hsapiens_gene_ensembl', host='http://www.ensembl.org')
##with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
##    display(dataset.list_attributes())
## or check attributes here: https://www.ensembl.org/biomart/martview
#
#gene_annot = dataset.query(attributes=["ensembl_gene_id", "external_gene_name", "hgnc_symbol", "uniprot_gn_symbol", "wikigene_name"])
#gene_annot
#
#gene_annot_2 = dataset.query(attributes=["ensembl_gene_id", "external_gene_name", "hgnc_id"])
#hgnc = pd.read_csv("group-471.csv", skiprows=1).rename(columns={'HGNC ID (gene)':'HGNC ID'})
#gene_annot_2 = gene_annot_2.set_index("HGNC ID").join(hgnc, how="left")
#gene_annot_2

# bar foo
#print(np.array([s in gene_annot["Gene name"].to_numpy() for s in barcodes.loc[~barcodes.matched, :].Primary.to_numpy()]).sum())
#print(np.array([s in gene_annot["HGNC symbol"].to_numpy() for s in barcodes.loc[~barcodes.matched, :].Primary.to_numpy()]).sum())
#print(np.array([s in gene_annot["UniProtKB Gene Name symbol"].to_numpy() for s in barcodes.loc[~barcodes.matched, :].Primary.to_numpy()]).sum())
#print(np.array([s in gene_annot["WikiGene name"].to_numpy() for s in barcodes.loc[~barcodes.matched, :].Primary.to_numpy()]).sum())
#
#print(np.array([s in gene_annot_2["Approved symbol"].to_numpy() for s in barcodes.loc[~barcodes.matched, :].Primary.to_numpy()]).sum())
#print(np.array([s in gene_annot_2["Approved name"].to_numpy() for s in barcodes.loc[~barcodes.matched, :].Primary.to_numpy()]).sum())
#print(np.array([s in gene_annot_2["Previous symbols"].to_numpy() for s in barcodes.loc[~barcodes.matched, :].Primary.to_numpy()]).sum())
#print(np.array([s in gene_annot_2["Aliases"].to_numpy() for s in barcodes.loc[~barcodes.matched, :].Primary.to_numpy()]).sum())

In [5]:
prot_path = "/Users/pschafer/Liana_Vignette/data/GSE213264/GSM6578071_humantonsil_protein.tsv.gz"
ab_path = "/Users/pschafer/Liana_Vignette/data/41587_2023_1676_MOESM3_ESM.xlsx"

In [10]:
ab_df = pd.read_excel(ab_path, sheet_name="HUMAN_273")
prot = pd.read_csv(prot_path, sep="\t")
prot = prot.set_index("X")

In [11]:
# remove unmapped from protein?
print(prot.shape)
unmapped_bool = np.array(["unmapped" in c for c in prot.columns])
print(unmapped_bool.sum())
prot = prot.loc[:,~unmapped_bool]
print(prot.shape)

(2492, 283)
1
(2492, 282)


In [12]:
# clean protein names
cleaned_names = prot.columns
cleaned_names = [re.sub("\\.[AGCT]{15}", "", s) for s in prot.columns]
cleaned_names = [re.sub("anti\\.mouse\\.human", "", s) for s in cleaned_names]
cleaned_names = [re.sub("anti\\.human\\.mouse", "", s) for s in cleaned_names]
cleaned_names = [re.sub("anti\\.mouse\\.rat\\.human", "", s) for s in cleaned_names]
cleaned_names = [re.sub("anti\\.mouse\\.rat", "", s) for s in cleaned_names]
cleaned_names = [re.sub("Mouse", "", s) for s in cleaned_names]
cleaned_names = [re.sub("Rat", "", s) for s in cleaned_names]
cleaned_names = [re.sub("Human", "", s) for s in cleaned_names]
cleaned_names = [re.sub("isotype\\.Ctrl", "", s) for s in cleaned_names]
cleaned_names = [re.sub("Isotype\\.Ctrl", "", s) for s in cleaned_names]
cleaned_names = [re.sub("\\.\\.k", "", s) for s in cleaned_names]
cleaned_names = [re.sub("\\.+$", "", s) for s in cleaned_names]
cleaned_names = [re.sub("^\\.+", "", s) for s in cleaned_names]

In [13]:
# merge from barcode excel sheet and clean names gain
barcodes = np.array([re.search("[AGCT]{15}|unmapped", s).group() for s in prot.columns])
barcodes = pd.DataFrame({"barcode": barcodes}).join(ab_df.set_index("Barcode"), on="barcode")
barcodes["Clean"] = cleaned_names
barcodes["Target"] = [re.sub("anti-human ", "", s) for s in barcodes["Target"].to_numpy()]
barcodes["Target"] = [re.sub("anti-human/mouse ", "", s) for s in barcodes["Target"].to_numpy()]
barcodes["Target"] = [re.sub("anti-mouse/human ", "", s) for s in barcodes["Target"].to_numpy()]
barcodes["Target"] = [re.sub("Recombinant", "", s) for s in barcodes["Target"].to_numpy()]
barcodes["Target"] = [re.sub("anti-Human", "", s) for s in barcodes["Target"].to_numpy()]
barcodes["Target"] = [re.sub("anti-", "", s) for s in barcodes["Target"].to_numpy()]
barcodes["Target"] = [s.strip() for s in barcodes["Target"].to_numpy()]

In [14]:
# get the primary target names and the alternative names in the brackets
def extract_names(s):
    pattern = re.compile(r'\((.*?)\)')
    return pattern.findall(s)

def remove_bracket(s):
    pattern = re.compile(r' \((.*?)\)')
    return pattern.sub('', s)

primary_names = []
alternative_names = []
for bc, target in zip(barcodes.barcode, barcodes.Target):
    primary_names.append(remove_bracket(target))
    alternative_names.append(extract_names(target))

barcodes["Primary"] = [s.strip() for s in primary_names]
barcodes["Alternative"] = alternative_names
barcodes = barcodes.set_index("Primary").join(cd_to_ncbi.set_index("CD_NAME"), how="left")
barcodes = barcodes.reset_index().rename(columns={"index": "Primary"})
barcodes = barcodes.fillna("none")

In [15]:
# get ligand-receptor information
#li.rs.show_resources()
db = li.rs.select_resource("consensus")
ligands_orig = np.unique(db.ligand.to_numpy())
receptors_orig = np.unique(db.receptor.to_numpy())
print(ligands_orig.shape)
print(receptors_orig.shape)

# NOTE: for now we remove entries that are annotated as both receptors and ligands
ligands = np.array(list(set(ligands_orig) - (set(ligands_orig) & set(receptors_orig))))
receptors = np.array(list(set(receptors_orig) - (set(ligands_orig) & set(receptors_orig))))
print(ligands.shape)
print(receptors.shape)

# remove ligand and receptors that are not in our rna data


(1036,)
(1059,)
(974,)
(997,)


In [16]:
# custom list to translate names
translate_names = {
    "CD270": "TNFRSF14",
    "CD335": "NCR1",
    "CD117": "KIT",
    "CD357": "TNFRSF18",
    "CD268": "TNFRSF13C",
    "CD252": "TNFSF4",
    "CD137L": "TNFSF9", # ligand for CD137: TNFRSF9
    "TNF-α" : "TNF",
    "LOX-1": "OLR1",
    "TSLPR": "CRLF2",
    "CD273": "PDCD1LG2",
    "CD169": "SIGLEC1",
    "β2-microglobulin": "B2M",
    "FcεRIα": "FCER2",
    "CD294": "PTGDR2",
    "integrin β7": "ITGB7",
    "CD32": "FCGR2A",
    "CD85j": "LILRB1",
    "CD158": "CD40LG",
    "Mac-2": "LGALS3",
    "Podoplanin": "PDPN",
    "CD8": "CD8A",
    "CD122": "IL2RB",
    "Galectin-9": "LGALS9",
    "Cadherin 11": "CDH11",
    "Notch 1": "NOTCH1",
    "CD119": "IFNGR1",
    "Notch 3": "NOTCH3",
    "CD85g": "LILRA4",
    "TIM-4": "TIMD4",
    "C5L2": "C5AR2",
    "mast cell tryptase": "TPSAB1",
    "GP130": "IL6ST",
    "CD218a": "IL18R1",
    "VEGFR-3": "FLT4",
    "GPR56": "ADGRG1",
    "MICA/MICB": "MICA",
    "NKp80": "KLRF1",
    "CD131": "CSF2RB",
    "Lymphotoxin β Receptor": "LTBR",
    "Annexin A1": "ANXA1",
}
barcodes["MANUAL"] = [translate_names.get(s, "none") for s in barcodes.Primary]

In [17]:
match_bool_receptors = np.zeros(len(barcodes), dtype=bool)
matched_receptors = []

for i, row_dict in enumerate(barcodes.to_dict(orient="records")):
    
    if row_dict["Primary"] in receptors:
        matched_receptors.append(row_dict["Primary"])
        match_bool_receptors[i] = True
        continue

    if row_dict["NCBI_NAME"] in receptors:
        matched_receptors.append(row_dict["NCBI_NAME"])
        match_bool_receptors[i] = True
        continue

    if row_dict["MANUAL"] in receptors:
        matched_receptors.append(row_dict["MANUAL"])
        match_bool_receptors[i] = True
        continue

    tmp_flag = False
    for entry in row_dict["NCBI_OTHER_NAME"]:
        if entry.strip() in receptors:
            matched_receptors.append(entry.strip() )
            match_bool_receptors[i] = True
            tmp_flag = True
            break
    if tmp_flag:
        continue

    tmp_flag = False
    for entry in row_dict["Alternative"]:
        if entry.strip() in receptors:
            matched_receptors.append(entry.strip() )
            match_bool_receptors[i] = True
            tmp_flag = True
            break
    if tmp_flag:
        continue

    matched_receptors.append("none")

barcodes["matched_receptor_bool"] = match_bool_receptors
match_bool_receptors.sum()

133

In [18]:
match_bool_ligands = np.zeros(len(barcodes), dtype=bool)
matched_ligands = []

for i, row_dict in enumerate(barcodes.to_dict(orient="records")):
    
    if row_dict["Primary"] in ligands:
        matched_ligands.append(row_dict["Primary"])
        match_bool_ligands[i] = True
        continue

    if row_dict["NCBI_NAME"] in ligands:
        matched_ligands.append(row_dict["NCBI_NAME"])
        match_bool_ligands[i] = True
        continue

    if row_dict["MANUAL"] in ligands:
        matched_ligands.append(row_dict["MANUAL"])
        match_bool_ligands[i] = True
        continue

    tmp_flag = False
    for entry in row_dict["NCBI_OTHER_NAME"]:
        if entry.strip() in ligands:
            matched_ligands.append(entry.strip() )
            match_bool_ligands[i] = True
            tmp_flag = True
            break
    if tmp_flag:
        continue

    tmp_flag = False
    for entry in row_dict["Alternative"]:
        if entry.strip() in ligands:
            matched_ligands.append(entry.strip() )
            match_bool_ligands[i] = True
            tmp_flag = True
            break
    if tmp_flag:
        continue

    matched_ligands.append("none")

barcodes["matched_ligand_bool"] = match_bool_ligands
match_bool_ligands.sum()

52

In [19]:
barcodes["matched"] = np.logical_or(barcodes["matched_receptor_bool"], barcodes["matched_ligand_bool"])

barcodes["matched_receptor"] = matched_receptors
barcodes["matched_ligand"] = matched_ligands

matched_receptors = np.array(matched_receptors)[[s != "none" for s in matched_receptors]]
matched_ligands = np.array(matched_ligands)[[s != "none" for s in matched_ligands]]

assert set(list(matched_receptors)) & set(list(matched_ligands)) == set()

In [20]:
barcodes

,Primary,barcode,DNA_ID,Target,Clean,Alternative,NCBI_NAME,GENE_NAME,NCBI_OTHER_NAME,MANUAL,matched_receptor_bool,matched_ligand_bool,matched,matched_receptor,matched_ligand
0,CD64,AAGTATGCCCTACGA,A0162,CD64,CD64,[],none,none,none,none,False,False,False,none,none
1,CD1d,TCGAGTCGCTTATCA,A0164,CD1d,CD1d,[],CD1D,CD1D,[none],none,False,True,True,none,CD1D
2,CD270,TGATAGAAACAGACC,A0020,"CD270 (HVEM, TR2)",CD270..HVEM_.TR2,"[HVEM, TR2]",TNFRSF14,CD270,"[ATAR, CD270, ""herpesvirus entry mediator"", ...",TNFRSF14,True,False,True,TNFRSF14,none
3,CD319,AGTATGCCATGTCTT,A0830,CD319 (CRACC),CD319..CRACC,[CRACC],SLAMF7,CD319,"[19A, CRACC, CS1]",none,False,False,False,none,none
4,CD34,GCAGAAATCTCCCTT,A0054,CD34,CD34,[],CD34,CD34,[none],none,False,True,True,none,CD34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,CD37,ACAGTCACTGGGCAA,A0941,CD37,CD37,[],CD37,CD37,[GP52-40],none,False,False,False,none,none
278,TNF-α,CCTATGAACGTAACG,A0945,TNF-α,TNF.a,[],none,none,none,TNF,False,True,True,none,TNF
279,CD215,CATATTCCGCCGTAA,A0947,CD215 (IL-15Rα),CD215..IL.15Ra,[IL-15Rα],IL15RA,CD215,"[CD215, IL-15RA]",none,False,True,True,none,IL15RA
280,CD321,GACAGTACCGACACT,A0948,CD321,CD321,[],F11R,CD321,"[JAM, KAT, JAM1, JCAM, JAM-1, PAM-1]",none,False,True,True,none,F11R


In [23]:
barcode_to_match = {}
for row_dict in barcodes.to_dict(orient="records"):
    if row_dict["matched_receptor_bool"]:
        barcode_to_match[row_dict["barcode"]] = row_dict["matched_receptor"]
    elif row_dict["matched_ligand_bool"]:
        barcode_to_match[row_dict["barcode"]] = row_dict["matched_ligand"]
len(barcode_to_match)

184

In [24]:
barcode_to_match_df = pd.DataFrame.from_dict(barcode_to_match, orient='index')
barcode_to_match_df.to_csv('ab_matching/barcode_to_match.csv')
barcodes.to_csv('ab_matching/barcodes.csv', index=False)